## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [27]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [28]:
pd.__version__

'1.2.0'

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * estado_vacunacion_.csv , última versión del mismo con el acumulado de todos los informes publicados
  * estado_vacunacion_year-month-day.csv, con la versión del día en cuestión, solo con la información de ese día

In [29]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=7)
fecha_informe_str=fecha_informe.strftime('%Y%m%d')
fecha_informe_pandas = fecha_informe.strftime('%Y-%m-%d')
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en formato ods

In [30]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [5]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210108.ods


In [6]:
datos_row

,Unnamed: 0,Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Fecha de la última vacuna registrada (2)
0,Andalucía,140295,69445,0.494993,2021-01-07
1,Aragón,23715,9944,0.419313,2021-01-08
2,Asturias,23720,17843,0.752234,2021-01-08
3,Baleares,12840,4257,0.331542,2021-12-07
4,Canarias,27660,9572,0.346059,2021-01-08
5,Cantabria,11850,2334,0.196962,2021-01-08
6,Castilla y Leon,51390,28325,0.551177,2021-01-07
7,Castilla La Mancha,35550,12380,0.348242,2021-01-07
8,Cataluña,120545,38826,0.322087,2021-01-07
9,C. Valenciana,61225,15965,0.260759,2021-01-08


..copia para tratarlo

In [7]:
datos_hoy = datos_row.copy()

renombramos columnas, añadimos columna con `fecha_de_informe` y ordenamos

In [8]:
datos_hoy.columns=['ccaa','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada']

In [9]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [10]:
datos_hoy=datos_hoy[['date_pub','ccaa','Fecha de la ultima vacuna registrada','Dosis entregadas','Dosis administradas','% sobre entregadas','source_name','source']]

eliminamos la fila Totales

In [11]:
datos_hoy = datos_hoy.drop(datos_hoy.index[-1])

In [12]:
datos_hoy

,date_pub,ccaa,Fecha de la ultima vacuna registrada,Dosis entregadas,Dosis administradas,% sobre entregadas,source_name,source
0,2021-01-08,Andalucía,2021-01-07,140295,69445,0.494993,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,2021-01-08,Aragón,2021-01-08,23715,9944,0.419313,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,2021-01-08,Asturias,2021-01-08,23720,17843,0.752234,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,2021-01-08,Baleares,2021-12-07,12840,4257,0.331542,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,2021-01-08,Canarias,2021-01-08,27660,9572,0.346059,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,2021-01-08,Cantabria,2021-01-08,11850,2334,0.196962,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,2021-01-08,Castilla y Leon,2021-01-07,51390,28325,0.551177,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,2021-01-08,Castilla La Mancha,2021-01-07,35550,12380,0.348242,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,2021-01-08,Cataluña,2021-01-07,120545,38826,0.322087,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,2021-01-08,C. Valenciana,2021-01-08,61225,15965,0.260759,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [13]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [14]:
#datos_acumulados = pd.read_csv('csv/Libro1.csv',sep=';')

In [31]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [32]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,2021-01-04,Andalucía,140295,25809,0.183962,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,2021-01-04,Aragón,23715,2004,0.084503,2/1/21,2021-01-02,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,2021-01-04,Asturias,23720,9380,0.395447,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,2021-01-04,Baleares,8940,153,0.017114,28/12/20,2020-12-28,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,2021-01-04,Canarias,20835,4846,0.232589,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
71,2021-01-08,Murcia,25685,8958,0.348764,NaN,2021-01-08,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
72,2021-01-08,Navarra,11850,3703,0.312489,NaN,2021-01-07,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
73,2021-01-08,País Vasco,31605,9811,0.310426,NaN,2021-01-07,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
74,2021-01-08,Ceuta,1005,637,0.633831,NaN,2021-01-07,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [17]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [18]:
#aseguramos formato fecha para evitar inconsistencia en el merge
datos_acumulados['date_pub'] = pd.to_datetime(datos_acumulados['date_pub'],format='%Y-%m-%d')
datos_acumulados['Fecha de la ultima vacuna registrada'] = pd.to_datetime(datos_acumulados['Fecha de la ultima vacuna registrada'],format='%Y-%m-%d')

In [19]:
datos_acumulados['% sobre entregadas']=datos_acumulados['Dosis administradas']/datos_acumulados['Dosis entregadas']

In [20]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,2021-01-04,Andalucía,140295,25809,0.183962,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,2021-01-04,Aragón,23715,2004,0.084503,2/1/21,2021-01-02,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,2021-01-04,Asturias,23720,9380,0.395447,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,2021-01-04,Baleares,8940,153,0.017114,28/12/20,2020-12-28,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,2021-01-04,Canarias,20835,4846,0.232589,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
71,2021-01-08,Murcia,25685,8958,0.348764,NaN,2021-01-08,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
72,2021-01-08,Navarra,11850,3703,0.312489,NaN,2021-01-07,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
73,2021-01-08,País Vasco,31605,9811,0.310426,NaN,2021-01-07,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
74,2021-01-08,Ceuta,1005,637,0.633831,NaN,2021-01-07,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [21]:
datos_acumulados.iloc[0]['date_pub'].month

1

y volvemos a subirlo..

In [22]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [38]:
datos_acumulados[datos_acumulados['ccaa'].str.startswith('Castilla y')]

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
6,2021-01-04,Castilla y Leon,51390,10928,0.212648,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
25,2021-01-05,Castilla y Leon,51390,16430,0.319712,5/1/21,2021-01-04,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
44,2021-01-07,Castilla y Leon,51390,22103,0.430103,NaN,2021-01-06,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
63,2021-01-08,Castilla y Leon,51390,28325,0.551177,NaN,2021-01-07,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [40]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Asturias ',
       'Castilla y Leon ', 'Murcia '], dtype=object)